In [15]:
import pandas as pd 
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import numpy as np
import math
import pickle
from sklearn.preprocessing import OneHotEncoder
import requests

In [2]:
dt = datetime.now()
dt.hour

13

In [ ]:
#df_main = pd.read_csv("../data/df_main.csv")
#df_products = pd.read_csv("/Users/janikbischoff/Documents/Uni/MLog/Semester 1/FSIS/webshop/data/df_product_table.csv")


In [54]:
import sqlite3

def get_db_connection():
    conn = sqlite3.connect('/Users/janikbischoff/Documents/Uni/MLog/Semester 1/FSIS/webshop/database.db')
    conn.row_factory = sqlite3.Row
    return conn

conn = get_db_connection()
products = conn.execute('SELECT * FROM cart').fetchall()

conn.close()

In [31]:
with open('/Users/janikbischoff/Documents/Uni/MLog/Semester 1/FSIS/api/data/mappings/name_mapping.pickle', 'rb') as handle:
    name_mapping = pickle.load(handle)
with open('/Users/janikbischoff/Documents/Uni/MLog/Semester 1/FSIS/api/data/mappings/zip_customer_mapping.pickle', 'rb') as handle:
    zip_customer_mapping = pickle.load(handle)
with open('/Users/janikbischoff/Documents/Uni/MLog/Semester 1/FSIS/api/data/mappings/customer_state_mapping.pickle', 'rb') as handle:
    customer_state_mapping = pickle.load(handle)
with open('/Users/janikbischoff/Documents/Uni/MLog/Semester 1/FSIS/api/data/mappings/zip_seller_mapping.pickle', 'rb') as handle:
    zip_seller_mapping = pickle.load(handle)
with open('/Users/janikbischoff/Documents/Uni/MLog/Semester 1/FSIS/api/data/mappings/seller_state_mapping.pickle', 'rb') as handle:
    seller_state_mapping = pickle.load(handle)



In [32]:
KEY = "bf698b456ad4cebada870914b2021fab"
def construct_api_call(query):
    url = f"http://api.positionstack.com/v1/forward?access_key={KEY}&query={query}"
    r = requests.get(url)
    body = r.json()
    #body["data"][0]["latitude"]
    #body["data"][0]["longitude"]
    try: 
        return [body["data"][0]["latitude"], body["data"][0]["longitude"]]
    except:
        return np.nan

In [33]:
def get_manhatten_distance(customer, seller):
    R = 6371000.785 

    phi_1 = customer[0] * (math.pi / 180)
    phi_2 = seller[0] * (math.pi / 180)
    delta_phi = (seller[0] - customer[0]) * (math.pi / 180)
    delta_lamda = (seller[1] - customer[1]) * (math.pi / 180)

    a = math.sin(delta_phi / 2) * math.sin(delta_phi / 2) + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lamda/2) * math.sin(delta_lamda/2)
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    d = R * c
    
    return d / 1000

In [34]:
def get_model_params(products, location):
    """
        product_category_name	[x]
        customer_zip_code	[x]
        customer_state	[x]
        seller_zip_code	[x]
        seller_state	[x]
        order_weekday   [x]

        item_count	[x]
        price	[x]
        freight_value [x]	
        product_weight_g [x]	
        product_volume_cm^3 [x]	
        delivery_distance	[x]
        order_hour_of_day [x]
    """
    columns_num = ["item_count", "price", "freight_value", "product_weight_g", "product_volume_cm^3", "delivery_distance", "order_hour_of_day"]
    df_num = pd.DataFrame(columns=columns_num)
    columns_cat = ["product_category_name", "customer_zip_code", "customer_state", "seller_zip_code", "seller_state", "order_weekday"]
    df_cat = pd.DataFrame(columns=columns_cat)
    
    dt = datetime.now()
    order_weekday = dt.weekday()
    names = [row["product_name"] for row in products]
    customer_zip_code = location[0]
    customer_state = location[1]
    customer_city = location[2]
    seller_zip = 7070
    seller_city = "guarulhos"
    seller_state = "SP"
    
    names_labeled = [name_mapping[name] for name in names]
    customer_zip_code_labeled = zip_customer_mapping[customer_zip_code]
    customer_state_labeled = customer_state_mapping[customer_state]
    seller_zip_labeled = zip_seller_mapping[seller_zip]
    seller_state_labeled = seller_state_mapping[seller_state]

    for item in range(len(names)):
        row = {"product_category_name":names_labeled[item], "customer_zip_code":customer_zip_code_labeled, "customer_state":customer_state_labeled, "seller_zip_code":seller_zip_labeled, 
                "seller_state":seller_state_labeled, "order_weekday":order_weekday}
        add_row = pd.Series(row)
        
        df_cat = df_cat.append(add_row, ignore_index=True)
    df_labeled = df_cat

    c_0 = df_labeled[['product_category_name']].values
    c_0 = np.asarray(c_0).astype('float32')

    c_1 = df_labeled[['customer_zip_code']].values
    c_1 = np.asarray(c_1).astype('float32')

    c_2 = df_labeled[['customer_state']].values
    c_2 = np.asarray(c_2).astype('float32')

    c_3 = df_labeled[['seller_zip_code']].values
    c_3 = np.asarray(c_3).astype('float32')

    c_4 = df_labeled[['seller_state']].values
    c_4 = np.asarray(c_4).astype('float32')

    c_5 = df_labeled[['order_weekday']].values
    c_5 = np.asarray(c_5).astype('float32')

    item_count = len(products)
    prices = [row["product_price"] for row in products]
    weights = [row["product_weight_g"] for row in products]
    volumes = [row["product_volume_cm3"] for row in products]
    order_hour_of_day = dt.hour
    query_seller = f"{seller_zip}, {seller_city}"
    query_customer = f"{customer_zip_code}, {customer_city}"
    lat_lon_customer = construct_api_call(query_seller)
    lat_lon_seller = construct_api_call(query_customer)
    distance = get_manhatten_distance(lat_lon_customer, lat_lon_seller)
    for item in range(len(prices)):
        row = {"item_count":item_count, "price":prices[item], "freight_value":23, "product_weight_g":weights[item], 
                "product_volume_cm^3":volumes[item], "delivery_distance":distance, "order_hour_of_day":order_hour_of_day}
        add_row = pd.Series(row)
        
        df_num = df_num.append(add_row, ignore_index=True)
    
    #print(df_num)
    ss = StandardScaler()
    x_standardized = ss.fit_transform(df_num)
    X = x_standardized
    X = np.asarray(X).astype('float32')
    #print(X)
    
   
    return {"cat_input_0": c_0, "cat_input_1": c_1, "cat_input_2": c_2, "cat_input_3": c_3,"cat_input_4": c_4, "cat_input_5": c_5,"numeric_inputs": X}
    


In [47]:
print(len(products))

2


In [36]:
model = tf.keras.models.load_model("/Users/janikbischoff/Documents/Uni/MLog/Semester 1/FSIS/api/data/models/31days_all.h5")

In [37]:
pickle_in = open("/Users/janikbischoff/Documents/Uni/MLog/Semester 1/FSIS/api/data/y.pickle", "rb")
df_y = pickle.load(pickle_in)
y = df_y.to_numpy()

In [38]:
enc = OneHotEncoder(handle_unknown='ignore')
y = y.reshape(-1,1)
enc.fit(y)
target = enc.transform(y).toarray()

In [39]:
target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [63]:
location = [28013,"RJ","campos dos goytacazes"]
#location = [8215,"SP","sao paulo"]
inputs = get_model_params(products, location)

/var/folders/r6/_64zz3zd7f51md31zxq08wnw0000gn/T/ipykernel_42322/1540951238.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cat = df_cat.append(add_row, ignore_index=True)
/var/folders/r6/_64zz3zd7f51md31zxq08wnw0000gn/T/ipykernel_42322/1540951238.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cat = df_cat.append(add_row, ignore_index=True)
/var/folders/r6/_64zz3zd7f51md31zxq08wnw0000gn/T/ipykernel_42322/1540951238.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_num = df_num.append(add_row, ignore_index=True)
/var/folders/r6/_64zz3zd7f51md31zxq08wnw0000gn/T/ipykernel_42322/1540951238.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

In [64]:
predictions = model.predict(
    inputs
    )

In [65]:
predictions

array([[0.0000000e+00, 5.8119139e-14, 1.7184641e-10, 5.3685188e-08,
        4.0669011e-06, 7.4056315e-06, 6.7654262e-05, 7.3501629e-05,
        9.9056587e-04, 1.3502575e-01, 9.7665153e-02, 2.8944531e-01,
        4.2383406e-02, 4.7076669e-02, 1.3335454e-03, 2.3682502e-03,
        4.7577035e-02, 2.9649070e-01, 3.1440472e-03, 1.7491801e-02,
        1.2737321e-03, 2.3618997e-04, 5.3231151e-06, 2.0039151e-03,
        5.3140870e-03, 8.9565273e-03, 6.1075122e-04, 1.5840599e-06,
        4.1007882e-04, 1.4691510e-07, 9.2471837e-06, 3.3567041e-05],
       [0.0000000e+00, 3.0917547e-16, 1.0559834e-12, 1.9988429e-08,
        2.7693420e-06, 2.2500735e-05, 1.3085015e-04, 2.4735567e-05,
        1.0061805e-04, 1.3340817e-01, 1.9856870e-01, 5.1885498e-01,
        3.9035087e-03, 6.2125283e-03, 6.9636321e-06, 5.3377312e-06,
        6.4104106e-03, 1.2423170e-01, 3.3184758e-04, 3.0195594e-03,
        4.5494608e-06, 2.1747142e-06, 3.2679623e-08, 2.2364642e-04,
        2.0008897e-04, 4.3076021e-03, 2.0843425

In [66]:
check = predictions.round()
b = np.zeros_like(predictions)
b[np.arange(len(predictions)), predictions.argmax(1)] = 1

In [67]:
etas = enc.inverse_transform(b)

In [68]:
etas

array([[17],
       [11]])

In [61]:
etas.ravel()[0]

11